In [1]:
import os
from shutil import copy
import pandas as pd
from tqdm.notebook import tqdm

## Read data

In [3]:
data = pd.read_csv('/kaggle/input/walrus-data-train-added/train_walrus/train_data_added.csv')
data

## Data preparation

In [4]:
filenames = data.file_name.unique()
filenames

In [5]:
train_files = filenames[:-7]
val_files = filenames[-7:-2]
test_files = filenames[-2:]

## Split data to train, val, test

In [6]:
train = data[data.file_name.isin(train_files)]
train.to_csv('train.csv', index=False)
os.mkdir('train')
for i in train_files:
    copy(f'/kaggle/input/walrus-data-train-added/train_walrus/{i}', f'train/{i}')
    
val = data[data.file_name.isin(val_files)]
val.to_csv('val.csv', index=False)
os.mkdir('val')
for i in val_files:
    copy(f'/kaggle/input/walrus-data-train-added/train_walrus/{i}', f'val/{i}')

test = data[data.file_name.isin(test_files)]
test.to_csv('test.csv', index=False)
os.mkdir('test')
for i in test_files:
    copy(f'/kaggle/input/walrus-data-train-added/train_walrus/{i}', f'test/{i}')

## Install libs, requirements

In [7]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

In [8]:
!/opt/conda/bin/python3.7 -m pip install --upgrade pip

In [9]:
!pip3 install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio===0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [10]:
!pip show torch

In [11]:
# Install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html

In [12]:
# Install mmdetection
!pip install mmdet

In [13]:
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

## Check the installation

In [14]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(mmcv.__version__)
print(get_compiling_cuda_version())
print(get_compiler_version())

!ls

## Download model

In [15]:
!mkdir checkpoints
!wget -c http://download.openmmlab.com/mmdetection/v2.0/detectors/detectors_cascade_rcnn_r50_1x_coco/detectors_cascade_rcnn_r50_1x_coco-32a10ba0.pth \
      -O checkpoints/detectors_cascade_rcnn_r50_1x_coco-32a10ba0.pth

## Check the input images

In [16]:
import mmcv
import matplotlib.pyplot as plt

img = mmcv.imread('/kaggle/input/walrus-data-train-added/train_walrus/46.jpg')
plt.figure(figsize=(15, 10))
plt.imshow(mmcv.bgr2rgb(img))
plt.show()

## Custom WalrusDataset for training

In [17]:
import copy
import os.path as osp

import mmcv
import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

@DATASETS.register_module()
class WalrusDataset(CustomDataset):

    CLASSES = ['walrus']

    def load_annotations(self, ann_file):
        cat2label = {k: i for i, k in enumerate(self.CLASSES)}
        # load image list from file
        anno = pd.read_csv(ann_file)
        image_list = anno.file_name.unique()
    
        data_infos = []
        for image_id in image_list:
            filename = f'{self.img_prefix}/{image_id}'
            image = mmcv.imread(filename)
            height, width = image.shape[:2]
    
            data_info = dict(filename=f'{image_id}', width=width, height=height)
    
            
            image_anno = anno[anno.file_name == image_id]
            bbox_names = anno['class']
            bboxes = [[line.x_from, line.y_from, line.x_from+line.width, line.y_from+line.height] for line in image_anno.itertuples()]
    
            gt_bboxes = []
            gt_labels = []
    
            for bbox_name, bbox in zip(bbox_names, bboxes):
                gt_labels.append(cat2label[bbox_name])
                gt_bboxes.append(bbox)

            data_anno = dict(
                bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                labels=np.array(gt_labels, dtype=np.long))

            data_info.update(ann=data_anno)
            data_infos.append(data_info)

        return data_infos

## Create the training config

In [18]:
from mmcv import Config
cfg = Config.fromfile('./configs/detectors/detectors_cascade_rcnn_r50_1x_coco.py')

In [19]:
from mmdet.apis import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'WalrusDataset'
cfg.data_root = '/kaggle/working/'

cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 8

cfg.data.test.type = 'WalrusDataset'
cfg.data.test.data_root = '/kaggle/working/'
cfg.data.test.ann_file = 'test.csv'
cfg.data.test.img_prefix = 'test'

cfg.data.train.type = 'WalrusDataset'
cfg.data.train.data_root = '/kaggle/working/'
cfg.data.train.ann_file = 'train.csv'
cfg.data.train.img_prefix = 'train'

cfg.data.val.type = 'WalrusDataset'
cfg.data.val.data_root = '/kaggle/working/'
cfg.data.val.ann_file = 'val.csv'
cfg.data.val.img_prefix = 'val'

for i in cfg.model.roi_head.bbox_head:
    i['num_classes'] = 1
cfg.load_from = 'checkpoints/detectors_cascade_rcnn_r50_1x_coco-32a10ba0.pth'

cfg.work_dir = '/kaggle/working/model'

cfg.optimizer.lr = 0.002
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = [0]

cfg.device = "cuda"

cfg.runner.max_epochs = 12

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

## Train datector

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector, inference_detector, init_detector, show_result_pyplot


# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

In [ ]:
!ls ../../input/test-image/photo_2022-05-16_15-31-31.jpg

## Check model results

In [ ]:
img = mmcv.imread('../../input/test-image/photo_2022-05-16_15-31-31.jpg')
model.cfg = cfg
result = inference_detector(model, img)
show_result_pyplot(model, img, result)

In [ ]:
result

In [ ]:
!rm /kaggle/working/train.csv
!rm /kaggle/working/val.csv
!rm /kaggle/working/test.csv
!rm -rf /kaggle/working/train
!rm -rf /kaggle/working/val
!rm -rf /kaggle/working/test
!rm -rf /kaggle/working/mmdetection